<a href="https://colab.research.google.com/github/Stefano-t/bioinf-lab/blob/main/k562_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

!pip install crr_labels epigenomic_dataset \
 ucsc_genomes_downloader cache_decorator minepy &> /dev/null && echo "done"

In [ ]:
from cache_decorator import Cache
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from matplotlib.axes import Axes
from matplotlib.figure import Figure
from matplotlib.colors import ListedColormap, LogNorm
from multiprocessing import cpu_count, Pool
import matplotlib.pyplot as plt
from epigenomic_dataset import active_enhancers_vs_inactive_enhancers
import seaborn as sbn

# Retrive epigenomic data

In [ ]:
cell_line = "K562"
genome_assembly = "hg38"
window_size = 256


aeie_X, aeie_y = active_enhancers_vs_inactive_enhancers(
    cell_line=cell_line,
    assembly=genome_assembly,
    window_size=window_size,
    root="datasets",
    verbose=1,
    binarize=True
)

#promoters_epigenomes, promoters_labels = load_epigenomes(
#    cell_line = cell_line,
#    dataset = "fantom",
#    region = "promoters",
#    window_size = window_size,
#    binarize = True
#)

#enhancers_epigenomes, enhancers_labels = load_epigenomes(
#    cell_line = cell_line,
#    dataset = "fantom",
#    region = "enhancers",
#    window_size = window_size,
#    binarize = True
#)

# Retrive genomic sequence

In [ ]:
from ucsc_genomes_downloader import Genome

genome = Genome("hg38")

In [ ]:
def to_bed(data:pd.DataFrame)->pd.DataFrame:
    """Return bed coordinates from given dataset."""
    return data.reset_index()[data.index.names]

In [ ]:
aeie_X

In [ ]:
aeie_X.describe()

In [ ]:
aeie_y

In [ ]:
aeie_y.describe()

In [ ]:
epigenomes = {
    "promoters": promoters_epigenomes,
    "enhancers": enhancers_epigenomes,
}

labels = {
    "promoters": promoters_labels,
    "enhancers": enhancers_labels,
}

In [ ]:
def describe_dataset(
    X: pd.DataFrame,
    y: pd.DataFrame
):
    """Perform a quick report for some relevant information in the dataset.

    Parameters
    ---------- 
    X: pd.DataFrame
        The dataframe to describe.
    y: pd.DataFrame
        The labels to describe.
    """
    print(X.describe())
    print(X[:5])
    print(y.describe())
    print(y[:5])
    print("="*30, "Feature Sample ration", "="*30)
    print(f"Features/Samples ratio is: {X.shape[0] / X.shape[1]}")
    print("="*30, "NaN count", "="*30)
    print(f"Total NaN values: {X.isna().values.sum()}/{X.values.size}")
    print(f"Max NaN in a row: {X.isna().sum(axis=1).max()}/{X.shape[1]}")
    print(f"Max NaN in a feature: {X.isna().sum().max()}/{X.shape[0]}")   

In [ ]:
for region, X in epigenomes.items():
   print(f"{region}") 
   describe_dataset(X, labels[region])
   print("|"*80)

In [ ]:
def binarize(
    y: pd.DataFrame,
    cell_line: str
)-> pd.DataFrame:
    return y[cell_line].astype(int)

In [ ]:
from sklearn.feature_selection import VarianceThreshold

def drop_constant_features(X: pd.DataFrame) -> (pd.DataFrame, bool):
    """Return a DataFrame without constant features in in.

    Parameters
    ----------------
    X: pd.DataFrame
        The dataframe to process.

    Returns
    ----------------
    A tuple with the new DataFrame and a boolean value to point out if the
    new DataFrame is distinct from the input one.
    """
    constant_feature_selector = VarianceThreshold()
    new_X = constant_feature_selector.fit_transform(X)
    return (new_X, new_X.shape[1] != X.shape[1])

In [ ]:
def impute_with_median(X: pd.DataFrame) -> pd.DataFrame:
    return X.fillna(X.median())

In [ ]:
from sklearn.preprocessing import RobustScaler

def robust_scaler(X: pd.DataFrame) -> pd.DataFrame:
    return pd.DataFrame(
        RobustScaler().fit_transform(X.values),
        columns=X.columns,
        index=X.index
    )

In [ ]:
from typing import Tuple

def preprocess_dataset(
    X: pd.DataFrame,
    y: pd.DataFrame,
    region: str,
    cell_line: str,
    binary: bool=True
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    y_new = y
    if binary: 
        y_new[cell_line] = binarize(
            y_new,
            cell_line
        )
    X_new, ok = drop_constant_features(X)
    if not ok: 
        print(f"{region} doesn't have costant features.")
        X_new = X
    X_new = impute_with_median(X_new)
    return robust_scaler(X_new), y_new

In [ ]:
#for region, X in epigenomes.items():
#    epigenomes[region] = preprocess_dataset(X, region)
    
aeie_X, aeie_y = preprocess_dataset(
    aeie_X,
    aeie_y,
    "ae-ie",
    cell_line,
    binary=True
)

# Class balance

In [ ]:
axes = sbn.histplot(
    data=aeie_y,
    bins=3
)

axes.set_title("A-E I-E class balance")

counts = aeie_y[cell_line].value_counts()
print(f"Active-Inactive ratio: {counts[1]/counts[0]}")
print(f"Active fraction in dataset: {counts[1]/(counts[0]+counts[1])}")

# Verify output correlation

In [ ]:
from scipy.stats import pearsonr

uncorrelated_features_with_output = {
    region: set()
    for region in epigenomes
}

for region, X in epigenomes.items():
    print("="*20, region.upper(), "="*20)
    for feature in tqdm(
        X.columns,
        desc=f"Running Person test for {region}",
        dynamic_ncols = True,
        leave = False
    ):
        correlation, p_value = pearsonr(
            X[feature].values.ravel(),
            labels[region].values.ravel()
        )
        if p_value > 0.01:
            print(f"Feature {feature} has no correlation with the output.")
            uncorrelated_features_with_output[region].add(feature)

print(uncorrelated_features_with_output)

In [ ]:
from scipy.stats import spearmanr

for region, X in epigenomes.items():
    print("="*20, region.upper(), "="*20)
    for feature in tqdm(
        X.columns,
        desc= f"Running Spearman test for {region}",
        dynamic_ncols = True,
        leave = False
    ):
        correlation, p_value = spearmanr(
            X[feature].values.ravel(),
            labels[region].values.ravel()
        )
        if p_value > 0.01:
            print(f"Feature {feature} has no correlation with the output {correlation}.")
            uncorrelated_features_with_output[region].add(feature)

print(uncorrelated_features_with_output)

In [ ]:
from minepy import MINE

for region, X in epigenomes.items():
    for feature in tqdm(
        uncorrelated_features_with_output[region],
        desc = f"Running MINE test for {region}",
        dynamic_ncols = True,
        leave = False
    ):
        mine = MINE()
        mine.compute_score(
            X[feature].values.ravel(),
            labels[region].values.ravel()
        )
        score = mine.mic()
        if score >= 0.05:
            uncorrelated_features_with_output[region].remove(feature)
        else:
            print(region, feature, score)

print(uncorrelated_features_with_output)

In [ ]:
for region, X in epigenomes.items():
    epigenomes[region] = X.drop(columns=[
        c
        for c in uncorrelated_features_with_output[region]
        if c in X.columns                     
    ])

In [ ]:
for _, X in epigenomes.items():
    print(X.shape)

# Feature Correlation

In [ ]:
from scipy.stats import entropy
from typing import List

def features_correlation(
    X: pd.DataFrame,
    p_value_threshold: float = 0.01,
    correlation_threshold: float = 0.95
) -> List:
    scores = []

    for i, feature_1 in tqdm(
        enumerate(X.columns),
        total = len(X.columns),
        desc = f"Running tests for {region}",
        dynamic_ncols = True,
        leave = False
    ):
        for feature_2 in X.columns[i+1:]:
            correlation, p_value = pearsonr(
                X[feature_1].values.ravel(),
                X[feature_2].values.ravel()
            )
            scores.append((correlation, feature_1, feature_2))
    #        if p_value < p_value_threshold and np.abs(correlation) > correlation_threshold:
    #            print(f"High correlation (Pearson): {feature_1} with {feature_2} ({correlation})")
    #            if entropy(X[feature_1]) > entropy(X[feature_2]):
    #                high_correlation[region].add(feature_2)
    #            else:
    #                high_correlation[region].add(feature_1)
    
    return scores

In [ ]:
result = {}

for region, X in epigenomes.items():
    result[region] = features_correlation(X, region)

In [ ]:
scores = {
    region: sorted(score, 
                   key=lambda x: np.abs(x[0]), 
                   reverse=True)
    for region, score in result.items()
}

In [ ]:
for region, X in epigenomes.items():
    # take the first three elems, unwrap them and then merge togheter
    _, firsts, seconds = list(zip(*scores[region][:3]))
    columns = list(set(firsts+seconds))
    print(f"Most correlated features from {region} epigenomes")
    sbn.pairplot(pd.concat([
        X[columns],
        labels[region],
    ], axis=1), hue=labels[region].columns[0])
    plt.show()

# Feature selection

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel

for region, X in epigenomes.items():
    print(f"Feature selection for {region}")
    clf = ExtraTreesClassifier(
        n_estimators=50,
        max_depth=10
    )
    clf.fit(X, labels[region].values.ravel())
    model = SelectFromModel(clf, prefit=True)
    X_new = pd.DataFrame(
        model.transform(X),
        columns=X.columns[model.get_support()],
        index=X.index
    )
    print(f"new shape: {X_new.shape}")
    epigenomes[region] = X_new

# Data visualization

In [ ]:
from sklearn.decomposition import PCA

def pca_decomposition(
    X: np.ndarray,
    components: int = 2
) -> np.ndarray:
    return PCA(n_components=components).fit_transform(X)
#        index=X.index,
#        columns=X.columns,

In [ ]:
from MulticoreTSNE import MulticoreTSNE as TSNE

@Cache(
    cache_path="decomposition/tsne/{task}/{perplexity}_{dimensionality_threshold}_{_hash}.pkl.gz",
    args_to_ignore=["X"]
)
def tsne_decomposition(
    X:np.ndarray,
    perplexity:int,
    task:str,
    dimensionality_threshold:int=50,
) -> np.ndarray:
    if X.shape[1] > dimensionality_threshold:
        X = pca_decomposition(X, components=dimensionality_threshold)
    return TSNE(
        perplexity=perplexity,
        n_jobs=cpu_count(),
        random_state=42
    ).fit_transform(X)

In [ ]:
pca_dec = pca_decomposition(aeie_X.values)
_ = sbn.scatterplot(
    x=pca_dec.T[0],
    y=pca_dec.T[1],
    hue=aeie_y[cell_line]
)

In [ ]:
dec = tsne_decomposition(
    aeie_X.values, 
    perplexity=30,
    task="aeie",
    dimensionality_threshold=50
)
_ = sbn.scatterplot(
    x=dec.T[0],
    y=dec.T[1],
    hue=aeie_y[cell_line]
)

# Model creation

In [ ]:
from typing import Tuple
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, ReLU, Concatenate, Layer
from tensorflow.keras.layers import Conv1D, MaxPool1D, GlobalAveragePooling1D, GlobalMaxPool1D, Flatten
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
from extra_keras_metrics import get_complete_binary_metrics, get_standard_binary_metrics,get_minimal_multiclass_metrics
from keras_mixed_sequence import MixedSequence, VectorSequence
from tqdm.keras import TqdmCallback

In [ ]:
def build_fixed_ffnn(
    input_shape: int
)->Tuple[Model, Layer, Layer]:
    """Returns Feed-Forward Neural Network model for binary classification.
    
    Parameters
    -----------------------
    input_shape: int,
        Number of features in the input layer.
        
    Returns
    -----------------------
    Triple with model, input layer and output layer.
    """
    input_epigenomic_data = Input(shape=(input_shape,), name="epigenomic_data")
    hidden = Dense(16, activation="relu")(input_epigenomic_data)
    hidden = Dense(4, activation="relu")(hidden)
    hidden = Dense(2, activation="relu")(hidden)
    last_hidden_ffnn = hidden
    output_ffnn = Dense(1, activation="sigmoid")(last_hidden_ffnn)

    ffnn = Model(
        inputs=input_epigenomic_data,
        outputs=output_ffnn,
        name="BinaryFixedFFNN"
    )

    ffnn.compile(
        optimizer=SGD(
            learning_rate=0.5
        ),
        loss="binary_crossentropy",
        metrics=get_minimal_multiclass_metrics()
    )
    return ffnn, input_epigenomic_data, last_hidden_ffnn

In [ ]:
def build_fixed_cnn(
    window_size: int
)->Tuple[Model, Layer, Layer]:
    """Returns Convolutional Neural Network model for binary classification.
    
    Parameters
    -----------------------
    window_size: int,
        Size of the input genomic window.
        
    Returns
    -----------------------
    Triple with model, input layer and output layer.
    """
    input_sequence_data = Input(shape=(window_size, 4), name="sequence_data")
    hidden = Conv1D(64, kernel_size=5, activation="relu")(input_sequence_data)

    for _ in range(2):
        hidden = Conv1D(
            64,
            kernel_size=4,
            activation="relu",
        )(hidden)
        
    hidden = MaxPool1D(pool_size=2)(hidden)
    
    for _ in range(3):
        hidden = Conv1D(
            128,
            kernel_size=3,
            activation="relu",
        )(hidden)
        
    hidden = MaxPool1D(pool_size=2)(hidden)
        
    for _ in range(3):
        hidden = Conv1D(
            128,
            kernel_size=3,
            activation="relu",
        )(hidden)
    
    hidden = GlobalAveragePooling1D()(hidden)
    hidden = Dropout(rate=0.5)(hidden)
    hidden = Dense(10, activation="relu")(hidden)
    hidden = Dense(10, activation="relu")(hidden)
    last_hidden_cnn = Dropout(rate=0.5)(hidden)
    output_cnn = Dense(1, activation="sigmoid")(last_hidden_cnn)

    cnn = Model(
        inputs=input_sequence_data,
        outputs=output_cnn,
        name="BinaryClassificationCNN"
    )

    cnn.compile(
        optimizer="nadam",
        loss="binary_crossentropy",
        metrics=get_minimal_multiclass_metrics()
    )

    return cnn, input_sequence_data, last_hidden_cnn

In [ ]:
def get_ffnn_sequence(
    X: np.ndarray,
    y: np.ndarray,
    batch_size: int = 1024
) -> MixedSequence:
    """Returns sequence to train a FFNN model on epigenomic data.
    
    Implementative details
    -------------------------
    This sequence can be used for either binary classification or
    for regresssion, just change the y accordingly.
    
    Parameters
    -------------------------
    X: np.ndarray,
        The vector from where to extract the epigenomic data.
    y: np.ndarray,
        The values the model should predict.
    batch_size: int = 1024,
        The size of the batches to generate
    
    Returns
    --------------------------
    MixedSequence object to train a FFNN.
    """
    return MixedSequence(
        x={
            "epigenomic_data": VectorSequence(
                X,
                batch_size
            )
        },
        y=VectorSequence(
            y,
            batch_size=batch_size
        )
    )


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping


@Cache(
    cache_path=[
        "model_histories/{cell_line}/{task}/{model_name}/history_{_hash}.csv.xz",
        "model_performance/{cell_line}/{task}/{model_name}/performance_{_hash}.csv.xz",
    ],
    args_to_ignore=[
        "model", "training_sequence", "test_sequence"
    ]
)
def train_model(
    model: Model,
    model_name: str,
    task: str,
    cell_line: str,
    training_sequence: MixedSequence,
    test_sequence: MixedSequence,
    holdout_number: int,
    epochs: int
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Returns training history and model evaluations.
    
    Parameters
    ---------------------
    model: Model,
        The model to train.
    model_name: str,
        The model name.
    task: str,
        The name of the task.
    cell_line: str,
        Name of the considered cell line.
    training_sequence: MixedSequence,
        The training sequence.
    test_sequence: MixedSequence,
        The test sequence.
    holdout_number: int,
        The number of the current holdout.
        
    Returns
    ----------------------
    Tuple with training history dataframe and model evaluations dataframe.
    """
    history = pd.DataFrame(model.fit(
        train_sequence,
        validation_data=test_sequence,
        epochs=epochs,
        verbose=False,
        callbacks=[
            EarlyStopping(
                "loss",
                min_delta=1e-3,
                patience=2,
                mode="min"
            ),
        ]
    ).history)
    
    train_evaluation = dict(zip(model.metrics_names, model.evaluate(train_sequence, verbose=False)))
    test_evaluation = dict(zip(model.metrics_names, model.evaluate(test_sequence, verbose=False)))
    train_evaluation["run_type"] = "train"
    test_evaluation["run_type"] = "test"
    
    for evaluation in (train_evaluation, test_evaluation):
        evaluation["model_name"] = model_name
        evaluation["task"] = task
        evaluation["holdout_number"] = holdout_number
    
    evaluations = pd.DataFrame([
        train_evaluation,
        test_evaluation
    ])
    
    return history, evaluations

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit

n_splits = 10
test_size = 0.2
random_state = 1902

stratified_split = StratifiedShuffleSplit(
    n_splits = n_splits,
    test_size = test_size,
    random_state = random_state
)

regression_split = ShuffleSplit(
    n_splits = n_splits,
    test_size = test_size,
    random_state = random_state
)

In [ ]:
history_story = []
performance_story = []

for holdout_number, (train_indices, test_indices) in tqdm(
    enumerate(stratified_split.split(aeie_X, aeie_y)),
    total=n_splits,
    leave=False,
    desc="Computing holdouts"
):
    train_X, test_X = aeie_X.iloc[train_indices], aeie_X.iloc[test_indices]
    train_y, test_y = aeie_y.iloc[train_indices], aeie_y.iloc[test_indices]
    
    train_y = train_y.values.flatten()
    test_y = test_y.values.flatten()
    train_X = train_X.values
    test_X = test_X.values

    ffnn, input_epigenomic_data, last_hidden_ffnn = build_fixed_ffnn(train_X.shape[1])

    train_sequence = get_ffnn_sequence(train_X, train_y, batch_size=32)
    test_sequence = get_ffnn_sequence(test_X, test_y, batch_size=32)

    history, performance = train_model(
        ffnn,
        ffnn.name,
        "aeie",
        cell_line,
        train_sequence,
        test_sequence,
        holdout_number,
        epochs=64
    )
    del train_X, train_y, test_X, test_y
    history_story.append(history)
    performance_story.append(performance)

In [ ]:
performance_story = pd.concat(performance_story)

In [ ]:
performance_story

In [ ]:
_, axs = plt.subplots(ncols=2, figsize=(20, 10))
sbn.barplot(
    x="AUROC",
    y="holdout_number",
    data=performance_story,
    hue="run_type",
    orient="h",
    ax=axs[0],
    ci="sd"
)
axs[0].set_title("AUROC")

sbn.barplot(
    x="AUPRC",
    y="holdout_number",
    data=performance_story,
    hue="run_type",
    orient="h",
    ax=axs[1]
)
_ = axs[1].set_title("AUPRC")